# Trabalho 1 - Computação Gráfica
### Alunos:
* Pedro Augusto Ribeiro Gomes - 11819125
* Sofhia de Souza Gonçalves - 11735035



### Primeiro, importamos as bibliotecas que serão utilizadas:


In [225]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image

### Inicializando a janela

In [226]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
altura = 1600
largura = 1200
window = glfw.create_window(largura, altura, "Malhas e Texturas", None, None)
glfw.make_context_current(window)

### GLSL para Vertex Shader

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec3.
* Definindo matrizes Model, View e Projection que acumulam transformações geométricas 3D e permitem navegação no cenário.
* void main() é o ponto de entrada do nosso programa (função principal).
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso combinamos nossa variável vec2 com uma variável vec4. Além disso, modificamos nosso vetor com base nas transformações Model, View e Projection.

In [227]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        attribute vec3 normals;
        
       
        varying vec2 out_texture;
        varying vec3 out_fragPos;
        varying vec3 out_normal;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
            out_fragPos = vec3(  model * vec4(position, 1.0));
            out_normal = vec3( model *vec4(normals, 1.0));            
        }
        """

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [228]:
fragment_code = """

        // parametro com a cor da(s) fonte(s) de iluminacao
        uniform vec3 lightPos; // define coordenadas de posicao da luz
        vec3 lightColor = vec3(1.0, 1.0, 1.0);
        
        // parametros da iluminacao ambiente e difusa
        uniform float ka; // coeficiente de reflexao ambiente
        uniform float kd; // coeficiente de reflexao difusa
        
        // parametros da iluminacao especular
        uniform vec3 viewPos; // define coordenadas com a posicao da camera/observador
        uniform float ks; // coeficiente de reflexao especular
        uniform float ns; // expoente de reflexao especular
        


        // parametros recebidos do vertex shader
        varying vec2 out_texture; // recebido do vertex shader
        varying vec3 out_normal; // recebido do vertex shader
        varying vec3 out_fragPos; // recebido do vertex shader
        uniform sampler2D samplerTexture;
        
        
        
        void main(){
        
            // calculando reflexao ambiente
            vec3 ambient = ka * lightColor;             
        
            // calculando reflexao difusa
            vec3 norm = normalize(out_normal); // normaliza vetores perpendiculares
            vec3 lightDir = normalize(lightPos - out_fragPos); // direcao da luz
            float diff = max(dot(norm, lightDir), 0.0); // verifica limite angular (entre 0 e 90)
            vec3 diffuse = kd * diff * lightColor; // iluminacao difusa
            
            // calculando reflexao especular
            vec3 viewDir = normalize(viewPos - out_fragPos); // direcao do observador/camera
            vec3 reflectDir = normalize(reflect(-lightDir, norm)); // direcao da reflexao
            float spec = pow(max(dot(viewDir, reflectDir), 0.0), ns);
            vec3 specular = ks * spec * lightColor;             
            
            // aplicando o modelo de iluminacao
            vec4 texture = texture2D(samplerTexture, out_texture);
            vec4 result = vec4((ambient + diffuse + specular),1.0) * texture; // aplica iluminacao
            gl_FragColor = result;

        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [229]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [230]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [231]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [232]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [233]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [234]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Carregando Modelos (vértices e texturas) a partir de Arquivos

A função abaixo carrega modelos a partir de arquivos no formato WaveFront.


In [235]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    normals = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue


        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])

        ### recuperando vertices
        if values[0] == 'vn':
            normals.append(values[1:4])

        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            face_normals = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if(len(w) == 3):
                    face_normals.append(int(w[2]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, face_normals, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces
    model['normals'] = normals

    return model

In [236]:
glHint(GL_LINE_SMOOTH_HINT, GL_DONT_CARE)
glEnable(GL_TEXTURE_2D)
qtd_texturas = 10
textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

### A lista abaixo armazena todos os vertices carregados dos arquivos

In [237]:
vertices_list = []    
normals_list = []    
textures_coord_list = []

### Carregamos cada modelo e definimos funções para desenhá-los

In [238]:
modelo = load_model_from_file('../assets/ground_stone/ground1.obj')
ground1_start = len(vertices_list)
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo ground_stone.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo ground_stone.obj. Vertice final:',len(vertices_list))

ground1_size = len(vertices_list) - ground1_start

### carregando textura equivalente e definindo um id (buffer): 
load_texture_from_file(1,'../assets/ground_stone/stone.jpg')

Processando modelo ground_stone.obj. Vertice inicial: 0
Processando modelo ground_stone.obj. Vertice final: 12


In [239]:
## loading sky
modelo = load_model_from_file('../assets/sky/sky.obj')
sky_start = len(vertices_list)
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo sky.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Processando modelo sky.obj. Vertice final:',len(vertices_list))

sky_size = len(vertices_list) - sky_start

### carregando textura equivalente e definindo um id (buffer): 
load_texture_from_file(2,'../assets/sky/animecloud.png')

Processando modelo sky.obj. Vertice inicial: 12
Processando modelo sky.obj. Vertice final: 2892


In [240]:
modelo = load_model_from_file('../assets/ground_forest/ground2.obj')
ground2_start = len(vertices_list)
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo ground_forest.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo ground_forest.obj. Vertice final:',len(vertices_list))

ground2_size = len(vertices_list) - ground2_start

### carregando textura equivalente e definindo um id (buffer): 
load_texture_from_file(3,'../assets/ground_forest/grass.jpg')

Processando modelo ground_forest.obj. Vertice inicial: 2892
Processando modelo ground_forest.obj. Vertice final: 2904


In [241]:
# loading fence
modelo = load_model_from_file('../assets/fence/fence.obj')
fence_start = len(vertices_list)
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo fence.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Processando modelo fence.obj. Vertice final:',len(vertices_list))

fence_size = len(vertices_list) - fence_start

### carregando textura equivalente e definindo um id (buffer): 
load_texture_from_file(4,'../assets/fence/fence.jpg')

Processando modelo fence.obj. Vertice inicial: 2904
Processando modelo fence.obj. Vertice final: 4488


In [242]:
modelo = load_model_from_file('../assets/house/house.obj')
house_start = len(vertices_list)
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo house.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Processando modelo house.obj. Vertice final:',len(vertices_list))

house_size = len(vertices_list) - house_start

### carregando textura equivalente e definindo um id (buffer): 
load_texture_from_file(5,'../assets/house/textures/Cottage_Clean_Base_Color.png')

Processando modelo house.obj. Vertice inicial: 4488
Processando modelo house.obj. Vertice final: 17337


In [243]:
modelo = load_model_from_file('../assets/table/wood_table.obj')
table_start = len(vertices_list)
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo table.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Processando modelo table.obj. Vertice final:',len(vertices_list))

table_size = len(vertices_list) - table_start

### carregando textura equivalente e definindo um id (buffer): 
load_texture_from_file(7,'../assets/table/wood_table.png')

Processando modelo table.obj. Vertice inicial: 17337
Processando modelo table.obj. Vertice final: 17733


In [244]:
# loading chair
modelo = load_model_from_file('../assets/chair/chair.obj')
chair_start = len(vertices_list)
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo chair.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Processando modelo chair.obj. Vertice final:',len(vertices_list))

chair_size = len(vertices_list) - chair_start

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(8,'../assets/chair/chair.png')

Processando modelo chair.obj. Vertice inicial: 17733
Processando modelo chair.obj. Vertice final: 18813


In [245]:
# loading box
modelo = load_model_from_file('../assets/box/box.obj')
box_start = len(vertices_list)
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo box.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Processando modelo box.obj. Vertice final:',len(vertices_list))

box_size = len(vertices_list) - box_start

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(9,'../assets/box/box.jpg')

Processando modelo box.obj. Vertice inicial: 18813
Processando modelo box.obj. Vertice final: 18849


In [246]:
# loading raptor
modelo = load_model_from_file('../assets/raptor/raptor.obj')
raptor_start = len(vertices_list)
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo raptor.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Processando modelo raptor.obj. Vertice final:',len(vertices_list))

raptor_size = len(vertices_list) - raptor_start

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(10,'../assets/raptor/raptor.jpg')

Processando modelo raptor.obj. Vertice inicial: 18849
Processando modelo raptor.obj. Vertice final: 31302


In [247]:
modelo = load_model_from_file('../assets/slenderman/slenderman.obj')
slender_start = len(vertices_list)
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo slenderman.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
    for normal_id in face[2]:
        normals_list.append( modelo['normals'][normal_id-1] )
print('Processando modelo slenderman.obj. Vertice final:',len(vertices_list))

slender_size = len(vertices_list) - slender_start

### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(11,'../assets/slenderman/slenderman.png')

Processando modelo slenderman.obj. Vertice inicial: 31302
Processando modelo slenderman.obj. Vertice final: 47286


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar slots.

Nós agora vamos requisitar três slots.
* Um para enviar coordenadas dos vértices.
* Outros para enviar coordenadas de texturas.
* Outro para as coordenadas normais

In [248]:
# Request a buffer slot from GPU
buffer = glGenBuffers(3)


###  Enviando coordenadas de vértices para a GPU

In [249]:
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

###  Enviando coordenadas de textura para a GPU

In [250]:
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

###  Enviando coordenadas de normais para a GPU

In [251]:
normals = np.zeros(len(normals_list), [("position", np.float32, 3)]) # três coordenadas
normals['position'] = normals_list


# Upload coordenadas normals de cada vertice
glBindBuffer(GL_ARRAY_BUFFER, buffer[2])
glBufferData(GL_ARRAY_BUFFER, normals.nbytes, normals, GL_STATIC_DRAW)
stride = normals.strides[0]
offset = ctypes.c_void_p(0)
loc_normals_coord = glGetAttribLocation(program, "normals")
glEnableVertexAttribArray(loc_normals_coord)
glVertexAttribPointer(loc_normals_coord, 3, GL_FLOAT, False, stride, offset)

### Desenhando nossos modelos
* Cada modelo tem um Model para posicioná-los no mundo.
* É necessário saber qual a posição inicial e total de vértices de cada modelo.
* É necessário indicar qual o ID da textura do modelo.

In [252]:
def draw_model(position, rotation, scale, start, model_size, textures, is_sky = False):
    # aplica a matriz model
    
    # rotacao
    angle = rotation[0];
    r_x = rotation[1]; r_y = rotation[2]; r_z = rotation[3];
    
    # translacao
    t_x = position[0]; t_y = position[1]; t_z = position[2];
    
    # escala
    s_x = scale[0]; s_y = scale[1]; s_z = scale[2];
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
     #### define parametros de ilumincao do modelo
    ka = 0.7 # coeficiente de reflexao ambiente do modelo
    kd = 0.7 # coeficiente de reflexao difusa do modelo
    ks = 0 # coeficiente de reflexao especular do modelo
    ns = 0.9 # expoente de reflexao especular

    # if is sky texture
    if is_sky == True: 
        kd = 0
    
    loc_ka = glGetUniformLocation(program, "ka") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka) ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # recuperando localizacao da variavel kd na GPU
    glUniform1f(loc_kd, kd) ### envia kd pra gpu    
    
    loc_ks = glGetUniformLocation(program, "ks") # recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks) ### envia ks pra gpu        
    
    loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) ### envia ns pra gpu        

    for texture_id in textures:
            
        #define id da textura do modelo
        glBindTexture(GL_TEXTURE_2D, texture_id)
        
        # desenha o modelo
        glDrawArrays(GL_TRIANGLES, start, model_size) ## renderizando

### Eventos para modificar a posição da câmera.

* Usei as teclas A, S, D e W para movimentação no espaço tridimensional.
* Usei a posição do mouse para "direcionar" a câmera.
* Usei as teclas I, K, J e L para movimentação de translação do slender
* Usei as teclas das setas de subir e descer para rotacionar o slender
* Usei as teclas das setas de direita e esquerda para aumentar e diminuir a escala do slender

In [253]:
cameraPos   = glm.vec3(0.0,  0.0,  1.0);
cameraFront = glm.vec3(0.0,  0.0, -1.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);

slenderRotation = 90.0 
slenderScale = 0.7
slenderX = 10.0
slenderY = -30.0


polygonal_mode = False
def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, polygonal_mode, ka_inc, slenderRotation, slenderScale, slenderX, slenderY
    
    cameraSpeed = 1
    newCameraPos = glm.vec3(cameraPos.x, cameraPos.y, cameraPos.z)
    if key == 87 and (action==1 or action==2): # tecla W
        newCameraPos += cameraSpeed * cameraFront
    
    if key == 83 and (action==1 or action==2): # tecla S
        newCameraPos -= cameraSpeed * cameraFront
    
    if key == 65 and (action==1 or action==2): # tecla A
        newCameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 68 and (action==1 or action==2): # tecla D
        newCameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed

    distance = np.linalg.norm(np.array(newCameraPos) - np.array([0.0, 0.0, 0.0]))
    if distance < 370:
        cameraPos.x = newCameraPos.x
        cameraPos.y = newCameraPos.y
        cameraPos.z = newCameraPos.z
    
    if key == 80 and action==1 and polygonal_mode==True: # tecla P
        polygonal_mode=False
    else:
        if key == 80 and action==1 and polygonal_mode==False: # tecla P
            polygonal_mode=True
    
    if key == 265 and (action==1 or action==2): # tecla seta pra cima
        slenderRotation += 2
        
    if key == 264 and (action==1 or action==2): # tecla seta pra baixo
        slenderRotation -= 2

    if key == 262 and (action==1 or action==2): # tecla seta pra direita
        slenderScale += 0.2
        
    if key == 263 and (action==1 or action==2): # tecla seta pra esquerda
        slenderScale -= 0.2

    if key == 73 and (action==1 or action==2): # tecla I
        slenderX += 0.2
    if key == 75 and (action==1 or action==2): # tecla K
        slenderX -= 0.2
    if key == 74 and (action==1 or action==2): # tecla J
        slenderY -= 0.2
    if key == 76 and (action==1 or action==2): # tecla L
        slenderY += 0.2
        
        
        
firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.3 
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;

    
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)


    
glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)

### Matrizes Model, View e Projection


In [254]:
def model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    
    angle = math.radians(angle)
    
    matrix_transform = glm.mat4(1.0) # instanciando uma matriz identidade

    
    # aplicando translacao
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))    
    
    # aplicando rotacao
    matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r_x, r_y, r_z))
    
    # aplicando escala
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))
    
    matrix_transform = np.array(matrix_transform)
    
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    cameraPos[1] = 5;
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 1000.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

### Nesse momento, exibimos a janela.

In [255]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

In [ ]:
glEnable(GL_DEPTH_TEST) ### importante para 3D
   

rotacao_inc = 0
while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    if polygonal_mode==True:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    if polygonal_mode==False:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)   
    

    # draw sky  
    draw_model([0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0], [0.75, 0.75, 0.75], sky_start, sky_size, [2], True)
    
    # draw house
    draw_model([-50.0, -3.0, -30.0], [90.0, -90.0, 1.0, 0.0], [5.0, 5.0, 5.0], house_start, house_size, [5])

    # draw table
    draw_model([-50.0, -0.5, -30.0], [-90.0, 1.0, 0.0, 0.0], [0.25, 0.25, 0.25], table_start, table_size, [7])
    # draw four chairs around the table
    draw_model([-50.0, 2.5, -26.0], [0.0, 0.0, 0.0, 1.0], [0.4, 0.4, 0.4], chair_start, chair_size, [8])
    draw_model([-48.0, 2.5, -30.0], [90.0, 0.0, 1.0, 0.0], [0.4, 0.4, 0.4], chair_start, chair_size, [8])
    draw_model([-50.0, 2.5, -34.0], [180.0, 0.0, 1.0, 0.0], [0.4, 0.4, 0.4], chair_start, chair_size, [8])
    draw_model([-52.0, 2.5, -30.0], [-90.0, 0.0, 1.0, 0.0], [0.4, 0.4, 0.4], chair_start, chair_size, [8])

    # draw box
    draw_model([-40.0, 0.0, -30.0], [-90.0, 1.0, 0.0, 0.0], [0.9, 0.9, 0.9], box_start, box_size, [9])

    # draw raptor
    draw_model([-10.0, -0.5, -30.0], [90.0, 1.0, 90.0, 0.0], [0.15, 0.15, 0.15], raptor_start, raptor_size, [10])

    # draw slenderman
    draw_model([slenderX, -0.5, slenderY], [slenderRotation, 1.0, 90.0, 0.0], [slenderScale, slenderScale, slenderScale], slender_start, slender_size, [11])

    # draw all grounds
    draw_model([0.0, -1.0, 0.0], [0.0, 0.0, 0.0, 1.0], [10.0, 10.0, 10.0], ground1_start, ground1_size, [1])
    draw_model([0.0, -1.0, 0.0], [0.0, 0.0, 0.0, 1.0], [10.0, 10.0, 10.0], ground2_start, ground2_size, [3])

    # draw fence dividing spaces
    for i in range(-400, -10, 10):
        draw_model([0.0, -1.1, i], [90.0, 0.0, 1.0, 0.0], [30.0, 30.0, 30.0], fence_start, fence_size, [4])
    for i in range(0, 400, 10):
        draw_model([0.0, -1.1, i], [90.0, 0.0, 1.0, 0.0], [30.0, 30.0, 30.0], fence_start, fence_size, [4])
    
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection)    
    
    

    
    glfw.swap_buffers(window)

glfw.terminate()